### 통계적 가설 검정

특정 가설이 사실인지 아닌지 검정(testing)해보고 싶은 경우가 있다.

- 가설(hypothesis)은 데이터 통계치에 대한 이야기로 변환될 수 있으며, 다양한 가정 하에서 특정 분포에 대한 확률변수의 관측치로 이해할 수 있다.
- 귀무가설(H0, null hypothesis): 기본적인 가설
- 대립가설(H1, alternative hypothesis): 비교하고 싶은 가설

보통 통계를 사용해서, 귀무가설 H0을 기각할지 말지를 결정한다.


### 예시: 동전 던지기

동전이 하나 있다. 이 동전이 공평한 동전인지, 아닌지를 검정하고 싶다.

동전에서 앞면이 나올 확률이 p라고 하면,
- 귀무가설(H0): p = 0.5 # 동전이 공평하다는 의미
- 대립가설(H1): p != 0.5

동전을 n번 던져서 앞면이 나온 횟수 X를 세는 것으로 검정을 진행한다.  

* 동전 던지기는 각각 베르누이 분포를 따를 것이고, 이 것은 X가 이항분포를 따르는 확률변수라는 것을 의미한다
* 이항분포는 정규분포로 근사한다

In [49]:
from __future__ import division
import math

def normal_approximation_to_binomial(n, p):
    """베르누이 시행의 평균과 표준편차를 계산한다"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

# 확률변수가 정규분포를 따른다는 가정 하에,
# normal_cdf를 사용하면 실제 동전 던지기로부터 얻은 값이 구간 안(혹은 밖)에 존재할 확률을 계산할 수 있다

def normal_cdf(x, mu=0, sigma=1):
    """정규분포의 누적분포함수"""
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

def normal_probability_below(hi, mu=0, sigma=1):
    """확률변수가 특정값보다 작을 확률"""
    return normal_cdf(hi, mu, sigma)

def normal_probability_above(lo, mu=0, sigma=1):
    """확률변수가 특정값보다 클 확률"""
    return 1 - normal_cdf(lo, mu, sigma)

def normal_probability_between(lo, hi, mu=0, sigma=1):
    """확률변수가 특정 범위 사이에 존재할 확률"""
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

def normal_probability_outside(lo, hi, mu=0, sigma=1):
    """확률변수가 특정 범위 밖에 존재할 확률"""
    return 1 - normal_probability_between(lo, hi, mu, sigma)

# 확률이 주어졌을 때 평균을 중심으로 하는 대칭 구간을 구할 수도 있다
def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    """이진 검색으로 역함수를 근사"""

    # 표준정규분포가 아니면 변환
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    
    low_z, low_p = -10.0, 0 # normal_cdf(-10)은 0에 근접
    hi_z, hi_p = 10.0, 1 # normal_cdf(10)은 1에 근접
    
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2 # 중간값
        mid_p = normal_cdf(mid_z) # 중간값의 누적분포
        if mid_p < p: # 중간값이 너무 작으면 더 큰 값을 검색
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            hi_z, hi_p = mid_z, mid_p
        else:
            break
            
    return mid_z # 확률변수 값을 반환

def normal_upper_bound(probability, mu=0, sigma=1):
    """P(Z <= z) = probability인 z 값을 반환"""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    """P(Z >= z) = probability인 z 값을 반환"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """입력한 probability를 포함하고, 평균을 중심으로 대칭인 구간"""
    tail_probability = (1 - probability) / 2
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    return lower_bound, upper_bound

print normal_probability_below(0) # 확률변수가 0보다 작을 확률
print normal_probability_below(0.5) # 확률변수가 0.5보다 작을 확률
print normal_probability_above(0.5) # 확률변수가 0.5보다 클 확률
print normal_probability_between(0, 0.5) # 확률변수가 0에서 0.5에 있을 확률
print normal_probability_outside(0, 0.5) # 확률변수가 0에서 0.5 바깥에 있을 확률

print normal_upper_bound(0.5) # 50% 확률인 확률변수 값
print normal_upper_bound(0.75) # 75% 확률인 확률변수 값
print normal_lower_bound(0.75) # 25% 확률인 확률변수 값
print normal_two_sided_bounds(0.5)


0.5
0.691462461274
0.308537538726
0.191462461274
0.808537538726
0.0
0.674486160278
-0.674486160278
(-0.6744861602783203, 0.6744861602783203)


In [50]:
# 실제로 동전을 1000번 던져 본다 (n = 1000, 동전이 공평하다면 p = 0.5)
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

print '평균:', mu_0
print '표준편차:', sigma_0



평균: 500.0
표준편차: 15.8113883008


#### 제1종 오류
- 비록 H0이 참이지만, H0을 기각하는 오류 (또는 대립가설 H1을 수용하는 오류)
- 예) 무죄인 사람을 유죄로 판결
- 보통 5%나 1%로 유의수준을 설정한다
- 유의수준(significance): 오류를 얼마나 허용해 줄 것인지를 의미하는 값

여기선, 1종 오류의 유의수준을 5%로 선택해보자.  
X가 주어진 범위를 벗어나면 귀무가설 H0을 기각하는 가설 검정을 고려해보자.

p가 정말로 0.5, 즉 공평한 동전이라는 귀무가설 H0이 참이라면,  
확률변수 X(동전 던지기 시행의 결과값)가 주어진 범위를 벗어날 확률은 5% 밖에 되지 않을 것이다.  

바꿔 말하면, 만약 H0이 참이라면, 이 가설 검정은 20번 중 19번은 올바른 결과를 줄 것이다.

In [51]:
# 1종 오류의 유의수준을 5%로 선택해보자
# X가 주어진 범위를 벗어나면 귀무가설 H0을 기각하는 가설 검정을 고려해보자

print normal_two_sided_bounds(0.95, mu_0, sigma_0) # 유의수준 5% 이내의 범위

(469.01026640487555, 530.9897335951244)


#### 제2종 오류
- H0이 참이 아닌데, H0을 참으로 수용하는 오류
- 즉, H0이 거짓이지만, H0을 기각하지 않는 오류
- 예) 유죄인 사람을 무죄로 판결

제2종 오류를 범하지 않을 확률을 구하면 검정력(power)을 알 수 있다.
- 검정력: 대립가설 H1이 참일 때(즉, 귀무가설 H0이 거짓일 때), 대립가설 H1을 수용하고 귀무가설 H0를 기각하는 옳은 결정을 할 확률

2종 오류를 측정하려면, 먼저 H0이 거짓이라는 것이 무엇을 의미하는지 알아볼 필요가 있다.  
(즉, 대립가설이 있어야 한다)

p가 0.5가 아니라는 말은 X의 분포에 대해 많은 것을 알려주지 않는다.  
예를 들어, p가 0.55, 즉 동전의 앞면이 나올 확률이 약간 편향되어 있다면 검정력은 다음과 같다.

In [66]:
# p가 0.5일 때, 유의수준이 5%인 구간
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

print 'p=0.5, 유의수준 5%일 때, 구간 시작:', lo
print 'p=0.5, 유의수준 5%일 때, 구간 끝:', hi

# p = 0.55인 경우의 실제 평균과 표준편차
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

print 'p=0.55일 때 실제 평균:', mu_1
print 'p=0.55일 때 실제 표준편차:', sigma_1

# 제2종 오류란 귀무가설(H0)을 기각하지 못한다는 의미
# 즉, X가 주어진 구간 안에 존재하는 경우를 의미
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
print '제2종 오류: p=0.55일 때, (p=0.5,유의수준 5%)인 구간이 존재할 확률:', type_2_probability

power = 1 - type_2_probability
print 'p=0.55일 때, 검정력:', power

p=0.5, 유의수준 5%일 때, 구간 시작: 469.010266405
p=0.5, 유의수준 5%일 때, 구간 끝: 530.989733595
p=0.55일 때 실제 평균: 550.0
p=0.55일 때 실제 표준편차: 15.7321327226
제2종 오류: p=0.55일 때, (p=0.5,유의수준 5%)인 구간이 존재할 확률: 0.113451998705
p=0.55일 때, 검정력: 0.886548001295


만약, p<=0.5, 즉 '동전이 앞면에 편향되어 있지 않다'를 귀무가설로 정한다면,  
X가 50보다 크면 귀무가설을 기각하고,  
50보다 작다면 기각하지 않는 단측검정이 필요해진다.
- 단측검정: 가설 검정에서 평균의 한쪽 끝값, 즉 표본 분포의 한쪽에 관심을 가지고 시행하는 검정 방법.


In [67]:
# 유의수준 5%인 가설검정을 위해, 분포의 95% 해당 값 이하인 경계 값을 찾는다
hi = normal_upper_bound(0.95, mu_0, sigma_0)

print '95% 경계값:', hi

# p=0.55일 때, (p=0.5,95% 경계값)의 누적분포확률
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
print '제2종 오류:', type_2_probability

power = 1 - type_2_probability
print '검정력:', power



95% 경계값: 526.007358524
제2종 오류: 0.0636205196693
검정력: 0.936379480331
